In [1]:
def add(a, b):
    return a + b


In [3]:
hasattr(add, "__get__"), hasattr(add, "__set__")  # function is a non-data descriptor

(True, False)

In [4]:
import sys

me = sys.modules["__main__"]
me

<module '__main__'>

In [5]:
f = add.__get__(None, me)
f is add

True

In [6]:
class Person:
    def __init__(self, name):
        self.name = name

    def say_hello(self):
        return f"{self.name} says hello"


In [7]:
Person.say_hello

<function __main__.Person.say_hello(self)>

In [8]:
p = Person("Alex")
p.say_hello

<bound method Person.say_hello of <__main__.Person object at 0x103702630>>

In [9]:
bound_method = Person.say_hello.__get__(p, Person)
bound_method

<bound method Person.say_hello of <__main__.Person object at 0x103702630>>

In [12]:
f1 = p.say_hello
f2 = p.say_hello
f1 is f2  # __get__ is called every time and it returns new bound method

False

In [15]:
p.say_hello(), bound_method()

('Alex says hello', 'Alex says hello')

In [16]:
type(bound_method)

method

In [18]:
bound_method.__func__, p.say_hello.__func__

(<function __main__.Person.say_hello(self)>,
 <function __main__.Person.say_hello(self)>)

In [24]:

def say_hello(self):
    return f"{self.name} says hello!"


class Person:
    def __init__(self, name):
        self.name = name


In [25]:
say_hello, say_hello.__get__  # regular function on a module level, expects one arg `self`

(<function __main__.say_hello(self)>,
 <method-wrapper '__get__' of function object at 0x103ad5580>)

In [28]:
import types
help(types.MethodType)  # method -> function that is bound to the function

Help on class method in module builtins:

class method(object)
 |  method(function, instance, /)
 |
 |  Create a bound instance method object.
 |
 |  Methods defined here:
 |
 |  __call__(self, /, *args, **kwargs)
 |      Call self as a function.
 |
 |  __delattr__(self, name, /)
 |      Implement delattr(self, name).
 |
 |  __eq__(self, value, /)
 |      Return self==value.
 |
 |  __ge__(self, value, /)
 |      Return self>=value.
 |
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |
 |  __gt__(self, value, /)
 |      Return self>value.
 |
 |  __hash__(self, /)
 |      Return hash(self).
 |
 |  __le__(self, value, /)
 |      Return self<=value.
 |
 |  __lt__(self, value, /)
 |      Return self<value.
 |
 |  __ne__(self, value, /)
 |      Return self!=value.
 |
 |  __reduce__(self, /)
 |      Helper for pickle.
 |
 |  __repr__(self, /)
 |      Return repr(self).
 |
 |  __setattr__(self, name, value, /)
 |      Implement setattr(self, name, value).
 |
 |  -------

In [30]:
class Person:
    def __init__(self, name):
        self.name = name

p = Person("Bob")
the_method = types.MethodType(say_hello, p)
the_method, the_method()

(<bound method say_hello of <__main__.Person object at 0x103d1c9b0>>,
 'Bob says hello!')

In [32]:
p, the_method.__func__

(<__main__.Person at 0x103d1c9b0>, <function __main__.say_hello(self)>)

In [35]:
class MyFunc:
    def __init__(self, func):
        self._func = func

    def __get__(self, instance, owner_class):
        if instance is None:
            print("__get__ called from the class")
            return self._func
        print("__get__ called from instance")
        return types.MethodType(self._func, instance)


def hello(self):
    print(f"{self.name} says hello from the func!")


class Person:
    def __init__(self, name):
        self.name = name

    say_hello = MyFunc(hello)

In [36]:
Person.say_hello

__get__ called from the class


<function __main__.hello(self)>

In [37]:
p = Person("Bob")
p.say_hello, p.say_hello()

__get__ called from instance
__get__ called from instance
Bob says hello from the func!


(<bound method hello of <__main__.Person object at 0x103b22d80>>, None)